v0.1

> TODO:
- input:
  - f"{data_clean_dir}/bg_fiction_all.tsv"
- output:
  - f'{data_processed_dir}/sent_fiction_nlp_features_part1_v1_checkpoint.tsv'
  - f'{data_processed_dir}/sent_fiction_nlp_features_part1_v1.tsv'
- df.copy(deep=True)
- remove the warning, start_time
- add descriptions + fix headings
- remove yellow
- extract all functions to py files
- run locally
- check typos

In [3]:
!pip install ufal.udpipe
# !pip install stanza
# !pip install spacy-stanza

In [4]:
# import numpy as np
import os
import pandas as pd
import polars as pl
import sys
import urllib.request
# import warnings
# import stanza
# import spacy_stanza

# import re
import time
import polars as pl

# from copy import copy, deepcopy
from ufal.udpipe import Model, Pipeline, ProcessingError

In [5]:
!if [ ! -f "/content/helpers/save_load_checkpoint_files.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/save_load_checkpoint_files.py; fi
!if [ ! -f "/content/helpers/parse_nlp_features.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/parse_nlp_features.py; fi
!if [ ! -f "/content/helpers/parse_nlp_morphtags.py" ]; then wget -P helpers/ https://raw.githubusercontent.com/MirkaIvanova/Projects/refs/heads/main/the-grammar-whisperer/helpers/parse_nlp_morphtags.py; fi


In [6]:
IS_GUEST = False
LOAD_SAVED_DATA = True

In [7]:
if IS_GUEST:
    root_dir = '.'
else:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    root_dir = "/content/drive/MyDrive/softuni/the-grammar-whisperer"

data_clean_dir = f"{root_dir}/data/clean"
data_processed_dir = f"{root_dir}/data/processed"
helpers_dir = f"{root_dir}/helpers"

if root_dir not in sys.path:
    sys.path.append(root_dir)
    sys.path.append(helpers_dir)

bg_fiction_clean_csv =  f"{data_clean_dir}/bg_fiction_all.tsv"

Mounted at /content/drive


In [25]:
from save_load_checkpoint_files import save_checkpoint, load_checkpoint
from parse_nlp_features import parse_nlp_features
from parse_nlp_features2 import parse_nlp_features2
from parse_nlp_morphtags import parse_nlp_morphtags

ModuleNotFoundError: No module named 'parse_nlp_features2'

In [9]:
# warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning) # 🧡

In [10]:
# pd.set_option('display.max_colwidth', 500)

## Add lemmas and pos tagging to fiction sentences

###### Load clean csv with sentences

In [11]:
start_time = time.time()
df_polars = pl.read_csv(bg_fiction_clean_csv, separator='\t')
df = df_polars.to_pandas()
del df_polars
df.shape

print(f"Execution time: {time.time() - start_time} seconds")

Execution time: 24.555294036865234 seconds


##### ➡️Split to words and lemmas using UDPipe

In [12]:
# Download the UDPipe Bulgarian model
model_path = "bulgarian-btb-ud-2.5-191206.udpipe"

if not os.path.exists(model_path):
    model_url_udpipe = "https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3131/bulgarian-btb-ud-2.5-191206.udpipe?sequence=6&isAllowed=y"
    urllib.request.urlretrieve(model_url_udpipe, model_path)

model = Model.load(model_path)
pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

###### Function to process one sentence into pos, features, etc.

In [13]:
# 💙mii move to py
def convert_udpipe_to_spacy(text):
    doc_str = pipeline.process(text)

    lines = doc_str.strip().splitlines()
    words = []
    lemmas = []
    spaces = []
    pos_tags = []
    morph_tags = []
    features = []
    dep_rels = []

    for line in lines:
        if line.startswith("#") or not line.strip():
            continue

        parts = line.split("\t")
        index, word, lemma, pos, tag, feats, head, dep_rel, _, misc = parts

        words.append(word)
        lemmas.append(lemma)
        pos_tags.append(pos)  # POS tag (simpler POS category)
        morph_tags.append(tag)  # Detailed morphological tag
        features.append(feats)
        dep_rels.append(dep_rel)

        if "SpaceAfter=No" in misc:
            spaces.append(False)
        else:
            spaces.append(True)

    return words, lemmas, pos_tags, morph_tags, features, dep_rels, len(words)

###### Process the entire clean dataframe in chunks using checkpoints

In [14]:
df = df.rename(columns={"text": "sentence"})

In [15]:
# 💙mii move to .py file
def find_unprocessed_indices(df, col):
    """Finds the indices of rows in the DataFrame where the column is NaN, assuming those rows are unprocessed."""

    # Check if the column exists
    if col not in df.columns:
        return df.index.tolist()

    # Find unprocessed rows where the value in the column is NaN
    unprocessed_mask = df[col].isna()
    unprocessed_indices = df[unprocessed_mask].index.tolist()

    return unprocessed_indices

def process_dataframe_with_checkpoints(_df, process_func, chunk_size=1000, output_path='final_output.csv', sep='.'):
    """
    Process a DataFrame in chunks with checkpoint capability using vectorized operations.

    Args:
        _df: Input DataFrame
        process_func: Function to apply to each chunk
        chunk_size: Number of rows to process in each chunk
        checkpoint_path: Path to save checkpoint files
        output_path: Path to save final output
    """

    df, checkpoint_filename = load_checkpoint(output_path, sep=sep)
    if df is None:
        df = _df.copy()
        print("Starting fresh processing...")
    else:
        print(f"Loaded checkpoint {checkpoint_filename} with {len(df)} rows")

    if False:
        if os.path.exists(checkpoint_path):
            df = pd.read_csv(checkpoint_path, sep=sep)
            print(f"Loaded checkpoint with {len(df)} rows")
        else:
            df = _df.copy()
            print("Starting fresh processing")

    # Find which rows are not processed yet
    # Assuming if 'words' column is NaN, the row hasn't been processed
    # unprocessed_mask = df['words'].isna()
    # unprocessed_indices = df[unprocessed_mask].index
    unprocessed_indices = find_unprocessed_indices(df, 'words')

    if len(unprocessed_indices) == 0:
        print("All rows already processed")
        df.to_csv(output_path, index=False, sep=sep)
        return df

    total_chunks = (df.shape[0] + chunk_size - 1) // chunk_size

    current_chunks = (len(unprocessed_indices) + chunk_size - 1) // chunk_size

    for current_chunk_num in range(current_chunks):
        try:
            chunk_start = current_chunk_num * chunk_size
            chunk_end = min((current_chunk_num + 1) * chunk_size, len(unprocessed_indices))
            current_indices = unprocessed_indices[chunk_start:chunk_end]
            start_idx = current_indices[0]
            end_idx = current_indices[-1]

            print(f"Processing chunk {current_chunk_num + 1}/{current_chunks} (indices [{start_idx}-{end_idx}]/{df.shape[0]})")

            # Process current chunk in a vectorized way
            chunk_sentences = df.loc[current_indices, 'sentence']
            results = chunk_sentences.apply(process_func).apply(pd.Series)

            # Update the dataframe with processed results
            results.columns = ["words", "lemmas", "pos", "morph", "features", "dep", "n_words"]
            df.loc[current_indices, results.columns] = results

            # Save checkpoint after each chunk
            latest_filename = save_checkpoint(output_path, df, sep=sep)
            print(f"Saved checkpoint {latest_filename}")

        except KeyboardInterrupt:
            print("\nProcessing interrupted. Progress saved in checkpoint file.")
            save_checkpoint(output_path, df, sep=sep)
            return df
        except Exception as e:
            print(f"Error processing chunk: {e}")
            save_checkpoint(output_path, df, sep=sep)
            raise

    # All processing complete, save final output
    unprocessed_indices = find_unprocessed_indices(df, 'words')
    if len(unprocessed_indices) == 0:
        print("Processing complete")
        df.to_csv(output_path, index=False, sep=sep)

    return df

###### Temporarily split into two parts to be processed in parallel

In [16]:
df.shape

(6690845, 1)

In [17]:
# split the df into two dataframes for parallel processing
df1 = df.iloc[:len(df)//2]
df2 = df.iloc[len(df)//2:]

print(f"df1 shape: {df1.shape}")
print(f"df2 shape: {df2.shape}")

df1 shape: (3345422, 1)
df2 shape: (3345423, 1)


In [18]:
chunk_size = 10000
output_path = f'{data_processed_dir}/sent_fiction_nlp_features_part1_v1.tsv'

if not LOAD_SAVED_DATA:
    df_pos_udpipe_part1 = process_dataframe_with_checkpoints(
        df1,
        convert_udpipe_to_spacy,
        chunk_size=chunk_size,
        output_path=output_path,
        sep = '\t'
    )
else:
    df_pos_udpipe_part1 = pd.read_csv(output_path, sep='\t')

###### Add gender, number, person

In [21]:
df_pos_udpipe_part1 = df_pos_udpipe_part1.drop(columns=['dep'])

In [24]:
# 16sec / 100,000 rows
# if not LOAD_SAVED_DATA:
start_time = time.time()
if 1: # ❤️mii
    results = df_pos_udpipe_part1.iloc[0:100000]["features"].apply(parse_nlp_features).apply(pd.Series)

    results.columns = ["gender", "number", "person"]
    df_pos_udpipe_part1[results.columns] = results

    df_pos_udpipe_part1.info()

print(f"Execution time: {time.time() - start_time} seconds")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3345422 entries, 0 to 3345421
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   sentence  object 
 1   words     object 
 2   lemmas    object 
 3   pos       object 
 4   morph     object 
 5   features  object 
 6   n_words   float64
 7   gender    object 
 8   number    object 
 9   person    object 
dtypes: float64(1), object(9)
memory usage: 255.2+ MB
Execution time: 16.110564470291138 seconds


###### Add case

In [22]:
# UDPIPE
output_path = f'{data_processed_dir}/sent_fiction_nlp_features_part1_v2.tsv'

if not LOAD_SAVED_DATA:
if 1:
    results = df_pos_udpipe_part1.iloc[0:10000]["morph"].apply(parse_nlp_morphtags).apply(pd.Series)

    results.columns = ["case"]
    df_pos_udpipe_part1[results.columns] = results
    df_pos_udpipe_part1.to_csv(output_path, index=False, sep='\t')
else:
    df_pos_udpipe_part1 = pd.read_csv(output_path)

df_pos_udpipe_part1.info()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/softuni/the-grammar-whisperer/data/processed/sent_fiction_nlp_features_part1_v2.tsv'

###### Convert to lowercase

In [ ]:
columns_to_lower = ['sentence', 'words', 'lemmas']

# UDPIPE
df_pos_udpipe_part1[columns_to_lower] = df_pos_udpipe_part1[columns_to_lower].apply(lambda x: x.str.lower())


###### Remove columns with repeated information

In [ ]:
# UDPIPE
if not LOAD_SAVED_DATA:
    df_pos_udpipe_part1_no_repeat = df_pos_udpipe_part1.drop(columns=['morph', 'features', 'dep'])

    output_path = f'{data_processed_dir}/sent_fiction_nlp_features_part1_v3_final.csv'
    df_pos_udpipe_part1_repeat.to_csv(output_path, index=False, sep='\t')
else:
    pass

In [ ]:
# STANZA
if not LOAD_SAVED_DATA:
    df_processed_stanza_no_repeat = df_processed_stanza.drop(columns=['morph', 'features', 'dep'])

    output_path = f'{data_processed_dir}/sent_wikipedia_nlp_features_stanza_final.csv'
    df_processed_stanza_no_repeat.to_csv(output_path, index=False)
else:
    pass

In [ ]:
!ls -alh {data_processed_dir}

total 11G
-rw------- 1 root root  76M Jan 21 19:07 bg_vocabulary_final.csv
-rw------- 1 root root  96M Jan 21 14:05 bg_vocabulary_v1.csv
-rw------- 1 root root 664M Jan 21 15:59 sent_wikipedia_nlp_features_final_10000_tmp.csv
-rw------- 1 root root 663M Jan 21 15:54 sent_wikipedia_nlp_features_final.csv
-rw------- 1 root root 100M Jan 20 09:50 sent_wikipedia_nlp_features.part01.rar
-rw------- 1 root root  56M Jan 20 09:50 sent_wikipedia_nlp_features.part02.rar
-rw------- 1 root root 608M Jan 23 18:38 sent_wikipedia_nlp_features_stanza_final_10000_tmp.csv
-rw------- 1 root root 591M Jan 23 18:46 sent_wikipedia_nlp_features_stanza_final.csv
-rw------- 1 root root 1.1G Jan 23 08:34 sent_wikipedia_nlp_features_stanza_v1_checkpoint80.csv
-rw------- 1 root root 1.1G Jan 23 08:35 sent_wikipedia_nlp_features_stanza_v1.csv
-rw------- 1 root root 1.2G Jan 23 15:50 sent_wikipedia_nlp_features_stanza_v2.csv
-rw------- 1 root root 1.3G Jan 21 11:11 sent_wikipedia_nlp_features_v1.csv
-rw------- 1 ro

###### Experiments

In [ ]:
df = pd.read_csv(f'{data_processed_dir}/sent_wikipedia_nlp_features_stanza_final.csv')
df1 = pd.read_csv(f"{data_processed_dir}/sent_wikipedia_nlp_features_stanza_final_10000_tmp.csv")

In [ ]:
df1.iloc[155].sentence

'то се наблюдава във всяка точка от вселената като идващо от всички посоки на космоса.'

In [ ]:
df1.iloc[155].pos

"['PRON', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'ADP', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT']"

In [ ]:
indices_to_filter = [111, 119, 126, 141, 144, 155, 175, 177, 181, 190, 192]

filtered_df = df[df.index.isin(indices_to_filter)]

filtered_df

,sentence,words,lemmas,pos,n_words,gender,number,person,case,word_count
111,в този сезон клубът поставя рекорд като не губ...,"['в', 'този', 'сезон', 'клубът', 'поставя', 'р...","['в', 'този', 'сезон', 'клуб', 'поставям-(се)'...","['ADP', 'DET', 'NOUN', 'NOUN', 'VERB', 'NOUN',...",19.0,.MMM.M....N.N..FFF.,.SSSSS..S.S.P..SSS.,....3...3..........,..if.i......d..dii.,19
119,раменете на кръста сочат като ръце към двата о...,"['раменете', 'на', 'кръста', 'сочат', 'като', ...","['раменете', 'на', 'кръст', 'соча', 'като', 'р...","['NOUN', 'ADP', 'NOUN', 'VERB', 'ADP', 'NOUN',...",10.0,F.M..F.MM.,P.SP.P.PC.,...3......,d.h..i....,10
126,"дебютните сезони на ""анталияспор"" са на границ...","['дебютните', 'сезони', 'на', '""', 'анталияспо...","['дебютен', 'сезон', 'на', '""', 'анталияспор',...","['ADJ', 'NOUN', 'ADP', 'PUNCT', 'PROPN', 'PUNC...",22.0,.M..M...F.N...MF...FF.,PP..S.P.S.S...SSS..SS.,......3.........3.....,di..i...d.d...i.....i.,22
141,"пророкът носи царска дреха, червена отворена к...","['пророкът', 'носи', 'царска', 'дреха', ',', '...","['пророк', 'нося-(се)', 'царски', 'дреха', ','...","['NOUN', 'VERB', 'ADJ', 'NOUN', 'PUNCT', 'ADJ'...",19.0,M.FF.FFF.F..FF.F.M.,SSSS.SSS.S..SS.S.P.,.3.................,f.ii.iii.d..ii...i.,19
144,защитава кандидатска дисертация през 1961 година.,"['защитава', 'кандидатска', 'дисертация', 'пре...","['защитавам', 'кандидатски', 'дисертация', 'пр...","['VERB', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', ...",7.0,.FF.FF.,SSS.SS.,3......,.ii..i.,7
155,"други автори смята, че нито политиката на гибе...","['други', 'автори', 'смята', ',', 'че', 'нито'...","['друг', 'автор', 'смятам', ',', 'че', 'нито',...","['ADJ', 'NOUN', 'VERB', 'PUNCT', 'SCONJ', 'CCO...",16.0,.M....F.M..M..F.,PPS...S.P..PPPP.,..3.........3...,ii....d.d..d..i.,16
175,"патос, посредством който данни и информация се...","['патос', ',', 'посредством', 'който', 'данни'...","['патос', ',', 'посредством', 'който', 'данна'...","['NOUN', 'PUNCT', 'ADP', 'PRON', 'NOUN', 'CCON...",15.0,M..MF.F...N..N.,S..SP.S.P.S.PP.,........3......,i...i.i...d.ii.,15
177,той е най-опитният и притежаващият най-много у...,"['той', 'е', 'най-опитният', 'и', 'притежаващи...","['аз', 'съм', 'опитен', 'и', 'притежавам', 'мн...","['PRON', 'AUX', 'ADJ', 'CCONJ', 'ADJ', 'ADV', ...",11.0,M.M.M.N.MM.,SSS.SPP.SS.,33.........,..f...i.ii.,11
181,тези първи следи са обикновено изработени каме...,"['тези', 'първи', 'следи', 'са', 'обикновено',...","['този', 'първи', 'следя', 'съм', 'обикновено'...","['DET', 'ADJ', 'NOUN', 'AUX', 'ADV', 'VERB', '...",18.0,..F...F.M.M.....N.,PPPP.PP.S.S..P.PP.,...3..............,..i...i.i.i..i.ii.,18
190,"по документи на цва - в.търново. софия, военно...","['по', 'документи', 'на', 'цва', '-', 'в.', 'т...","['по', 'документ', 'на', 'цва', '-', 'в.', 'тъ...","['ADP', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'PROP...",18.0,.M.F.MN.F.NN.F.NF.,.P.S.SS.S.SS.S.SS.,..................,.i.i.ii.i.ii...i..,18
